<a href="https://colab.research.google.com/github/Vincent-ondeng/Final_year_research_project/blob/main/Notebook/combined_webmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the neccessary libraries and dependencies
!pip install folium
!pip install geopandas
!pip install pandas
!pip install numpy

In [2]:
#Import Libraries
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
from folium.features import GeoJsonTooltip

In [33]:
# Read the GeoJSON file using Geopandas
geojson = gpd.read_file("/content/wards.geojson")
geojson=geojson[['wardcode','geometry']] #only select 'wardcode' (Ward_Code) and 'geometry' columns

geojson2 = gpd.read_file("/content/subcounty.geojson")
geojson2=geojson2[['FID_','geometry']] #only select 'FID' and 'geometry' columns

In [34]:
#Read the covid-19 data using pandas
malaria_df=pd.read_csv("/content/wards.csv")
subcounty_df=pd.read_csv("/content/subcounty.csv")

In [35]:
geojson.head()
geojson2.head()
malaria_df.head()
subcounty_df.head()

,FID_,Sub-county Name,Confirmed Malaria (only Positive cases) <5 yrs,Confirmed Malaria (only Positive cases) >5 yrs,Malaria in pregnancy >5 yrs,Total Confirmed Cases in All age groups,Malaria Incidence per 1000
0,1,Kisumu Central SubCounty,9633,33787,828,44248,262
1,2,Kisumu East Sub-County,9370,37132,778,47280,315
2,3,Kisumu West Sub County,8983,36614,422,46019,351
3,4,Muhoroni Sub-County,8787,38584,531,47902,329
4,5,Nyakach Sub County,7214,33788,591,41593,313


In [36]:
#Join the geojson file with malaria_df
df_final = geojson.merge(malaria_df, left_on="wardcode", right_on="wardcode", how="outer")

In [37]:
#Join the geojson file with malaria_df
df_final2 = geojson2.merge(subcounty_df, left_on="FID_", right_on="FID_", how="outer")

In [38]:
df_final.head()
df_final2.head()

,FID_,geometry,Sub-county Name,Confirmed Malaria (only Positive cases) <5 yrs,Confirmed Malaria (only Positive cases) >5 yrs,Malaria in pregnancy >5 yrs,Total Confirmed Cases in All age groups,Malaria Incidence per 1000
0,1,"POLYGON ((34.74603 -0.14904, 34.73881 -0.14870...",Kisumu Central SubCounty,9633,33787,828,44248,262
1,2,"MULTIPOLYGON (((34.75277 -0.03298, 34.75388 -0...",Kisumu East Sub-County,9370,37132,778,47280,315
2,3,"POLYGON ((34.60659 0.00821, 34.60749 0.00676, ...",Kisumu West Sub County,8983,36614,422,46019,351
3,4,"MULTIPOLYGON (((35.14042 -0.16027, 35.13952 -0...",Muhoroni Sub-County,8787,38584,531,47902,329
4,5,"POLYGON ((35.05211 -0.28057, 35.05194 -0.28192...",Nyakach Sub County,7214,33788,591,41593,313


In [ ]:
# Create a map instance
m = folium.Map(location=[-0.15386345206107715, 34.86414568574279],
               zoom_start=10,
               control_scale=True,
               prefer_canvas=True)
m

In [ ]:
folium.Choropleth(
    geo_data=geojson,
    name="Malaria Distribution by ward",
    data=malaria_df,
    columns=['wardcode', 'Total_co'],
    key_on='feature.properties.wardcode',
    nan_fill_color="White", #Use white color if there is no data available for the county
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Total Confirmed Malaria  Cases in Kisumu",
    line_color='black',
    highlight=True,
).add_to(m)
m

In [ ]:
#Add Customized Tooltips to the map
folium.features.GeoJson(
                    data=df_final,
                    name='Total Confirmed cases of Malaria in Kisumu',
                    smooth_factor=2,
                    style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                    tooltip=folium.features.GeoJsonTooltip(
                        fields=['wardcode',
                                'ward',
                                'Confirme',
                                'Confirm1',
                                'Malaria_',
                                'Total_co'
                               ],
                        aliases=["WardCode:",
                                 "Ward Name:",
                                 "Confirmed Cases in Children under agve 5:",
                                 "Confirmed Cases in persons greater than age 5:",
                                 "Confirmed Cases in expectant women:",
                                 "Total Confirmed Malaria Cases:"
                                ], 
                        localize=True,
                        sticky=False,
                        labels=True,
                        style="""
                            background-color: #F0EFEF;
                            border: 2px solid black;
                            border-radius: 3px;
                            box-shadow: 3px;
                        """,
                        max_width=800,),
                            highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                        ).add_to(m)   

m

In [ ]:
folium.Choropleth(
    geo_data=geojson2,
    name="Malaria Distribution by Sub-county",
    data=subcounty_df,
    columns=['FID_', 'Total Confirmed Cases in All age groups'],
    key_on='feature.properties.FID_',
    nan_fill_color="White", #Use white color if there is no data available for the county
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Total Confirmed Malaria  Cases in Kisumu",
    line_color='black',
    highlight=True,
).add_to(m)
m

In [ ]:
#Add Customized Tooltips to the map
folium.features.GeoJson(
                    data=df_final2,
                    name='Total Confirmed cases of Malaria in Kisumu',
                    smooth_factor=2,
                    style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                    tooltip=folium.features.GeoJsonTooltip(
                        fields=['Sub-county Name',
                                'Confirmed Malaria (only Positive cases) <5 yrs',
                                'Confirmed Malaria (only Positive cases) >5 yrs',
                                'Malaria in pregnancy >5 yrs',
                                'Total Confirmed Cases in All age groups',
                                'Malaria Incidence per 1000'
                               ],
                        aliases=["Sub-county Name:",
                                 "Confirmed Malaria (only Positive cases) <5 yrs:",
                                 "Confirmed Malaria (only Positive cases) >5 yrs:",
                                 "Malaria in pregnancy >5 yrs:",
                                 "Total Confirmed Cases in All age groups:",
                                 "Malaria Incidence per 1000:"
                                ], 
                        localize=True,
                        sticky=False,
                        labels=True,
                        style="""
                            background-color: #F0EFEF;
                            border: 2px solid black;
                            border-radius: 3px;
                            box-shadow: 3px;
                        """,
                        max_width=800,),
                            highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                        ).add_to(m)   

m